In [1]:
import matplotlib.animation as animation
from matplotlib import rc, rcParams
import matplotlib.pyplot as plt 
from IPython.display import HTML

from double_pendulum import Simulation, RK4, init, animate
from double_pendulum import acceleration_double_pendulum
    

state = [2., 2.5, 0., 0.]
M = [1., 1.]
L = [1., 1.]
kwargs = { "M": M, "L": L, 'trail_frames': 50}


plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['animation.embed_limit'] = 3*1e7

fig, ax = plt.subplots( figsize=(10,10) )
ax.set_xlim( -sum(L), sum(L) )
ax.set_ylim( -sum(L), sum(L) )

plots = init(ax)
sim = Simulation( state, acceleration_double_pendulum, RK4, plots, kwargs )

anim = animation.FuncAnimation(fig, animate, frames=1000,                       
            interval=30, blit=True, fargs=(0.0175, sim))
plt.close()

In [3]:
#HTML(anim.to_jshtml())
HTML(anim.to_html5_video())